In [ ]:
import os
import pandas as pd
from sklearn import model_selection

In [ ]:
df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
df['kfold'] = -1 # Create a column named kfold and fill it with random value, say -1

In [ ]:
df = df.sample(frac=1).reset_index(drop=True) # Shuffle the dataset

#Get target values
y = df.target.values
kf = model_selection.StratifiedKFold(n_splits=10)

In [ ]:
for fold_, (_, _) in enumerate(kf.split(X=df, y=y)):
    df.loc[:, "kfold"] = fold_
df.to_csv('train_folds.csv', index= False)

In [ ]:
import glob
from tqdm import tqdm
from PIL import Image, ImageFile
from joblib import Parallel, delayed

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def resize_image(image_path, output_folder, resize):
    base_name = os.path.basename(image_path)
    outpath = os.path.join(output_folder, base_name)
    img = Image.open(image_path)
    img = img.resize(
                    (resize[1], resize[0]),
                    resample = Image.BILINEAR)
    img.save(outpath)

In [ ]:
! mkdir /kaggle/working/train512
! mkdir /kaggle/working/test512

In [ ]:
#Resize train images
input_folder = '../input/siim-isic-melanoma-classification/jpeg/train'
output_folder = '/kaggle/working/train512'
images = glob.glob(os.path.join(input_folder, '*.jpg'))
Parallel(n_jobs=12)(
        delayed(resize_image)(
            i,
            output_folder,
            (512,512)
        ) for i in tqdm(images)
)

In [ ]:
# Resize test images
input_folder = '../input/siim-isic-melanoma-classification/jpeg/test'
output_folder = '/kaggle/working/test512'
images = glob.glob(os.path.join(input_folder, '*.jpg'))
Parallel(n_jobs=12)(
        delayed(resize_image)(
            i,
            output_folder,
            (512,512)
        ) for i in tqdm(images)
)

In [ ]:
!tar -zcvf test512.tar.gz /kaggle/working/test512
!tar -zcvf train512.tar.gz /kaggle/working/train512